In [6]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook

# Toggle for debugging: if True, only keep essential columns.
DEBUG_MODE = True

# Set file and sheet details
file_path = "mario_kart_stats - Backup3.xlsx"
sheet_name = "Builds Normalized"

# Load the dataset (assuming headers are on the second row)
df = pd.read_excel(file_path, sheet_name=sheet_name, header=1)

# Define the key stats and their corresponding weights as pairs (stat, weight)
stats_and_weights = [
    ("BrakeRt_SAND_H Normal", 0.0),   # Offroad Sand Speed Max Speed
    ("BrakeRt_ICE_H Normal", 0.0),    # Offroad Ice Speed Max Speed
    ("SlipRt_SAND_H Normal", -0.5),  # Offroad Sand Slip (lower is better)
    ("SlipRt_ICE_H Normal", -0.5),   # Offroad Ice Slip (lower is better)
    ("SMT Length Normal", 4),
    ("Drive_SpdMaxFwd_H Normal", 3),
    ("Float_SpdMaxFwd_H Normal", 0.5),
    ("SpdMaxFwd_H Normal", 0.5),
    ("Drift_DirRotH Normal", 1.0),
    ("Float_Drift_DirRotH Normal", 0.5),
    ("Drift_DirRotH2 Normal", 0.5),
    ("Invincibility Time Multiplier Normal", 0.5),
]

# Unzip the pairs into separate lists for easier processing
key_stats = [stat for stat, weight in stats_and_weights]
weights = np.array([weight for stat, weight in stats_and_weights])

# If in debug mode, retain only the essential columns
if DEBUG_MODE:
    keep_columns = ["Driver", "Body", "Tire", "Glider"] + key_stats
    df = df[keep_columns]

# --- 1. Compute a Weighted Sum Score ---
# Since the data is already normalized, we directly compute the dot product.
scores = df[key_stats].to_numpy().dot(weights)
df['Score'] = scores

# --- 2. Rank and Select Top Setups ---
top_n = 10
best_setups = df.sort_values(by='Score', ascending=False).head(top_n)

# --- 3. Display and Save Results ---
cols_to_display = ["Driver", "Body", "Tire", "Glider"] + key_stats + ['Score']
print("\n=== Best Trade-Off Kart Setups (Weighted Sum Approach) ===")
print(best_setups[cols_to_display])

# Save the results to a new Excel file
output_file = "best_tradeoff_kart_builds_weighted.xlsx"
best_setups[cols_to_display].to_excel(output_file, index=False)
print(f"\nResults saved to {output_file}")

# --- 4. Auto-Fit Column Widths using openpyxl ---
wb = load_workbook(output_file)
ws = wb.active

# Loop through each column and adjust the width based on the longest cell in that column.
for col in ws.columns:
    max_length = 0
    # Get the column letter (e.g., 'A')
    column = col[0].column_letter
    for cell in col:
        try:
            if cell.value:
                max_length = max(max_length, len(str(cell.value)))
        except:
            pass
    adjusted_width = max_length + 2  # Adding a little extra space
    ws.column_dimensions[column].width = adjusted_width

wb.save(output_file)
print("Column widths adjusted and file saved.")



=== Best Trade-Off Kart Setups (Weighted Sum Approach) ===
                                      Driver               Body  \
788                   Baby Peach, Baby Daisy       Tanooki Kart   
12116  Peach, Daisy, Yoshi, Birdo, Peachette           Streetle   
840                   Baby Peach, Baby Daisy        Koopa Clown   
892                   Baby Peach, Baby Daisy  Master Cycle Zero   
12792  Peach, Daisy, Yoshi, Birdo, Peachette       Tanooki Kart   
12168  Peach, Daisy, Yoshi, Birdo, Peachette           Landship   
792                   Baby Peach, Baby Daisy       Tanooki Kart   
12800  Peach, Daisy, Yoshi, Birdo, Peachette       Tanooki Kart   
12220  Peach, Daisy, Yoshi, Birdo, Peachette         Pipe Frame   
12896  Peach, Daisy, Yoshi, Birdo, Peachette  Master Cycle Zero   

                       Tire  \
788                 Cushion   
12116  Roller, Azure Roller   
840                 Cushion   
892                 Cushion   
12792  Roller, Azure Roller   
12168  Roller, A